<a href="https://colab.research.google.com/github/lamyse1/Recommender-Systems/blob/main/Deep_Learning_Recommendation_System_(Chapter_9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install recommenders -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 470.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.3/355.3 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 15.8 MB/s

In [16]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
#MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 5
BATCH_SIZE = 256

SEED = 42

##1. Load dataset

In [3]:
import pandas as pd

In [4]:
df = pd.read_excel('Rec_sys_data.xlsx')

In [5]:
df= df[['CustomerID','StockCode','Quantity','DeliveryDate']]

In [6]:
df["StockCode"] = pd.to_numeric(df["StockCode"], errors='coerce')

In [7]:
df = df.dropna()
print(df.shape)
df

(246706, 4)


,CustomerID,StockCode,Quantity,DeliveryDate
1,17850,71053.0,6,2010-12-02 08:26:00
2,17850,21730.0,6,2010-12-03 08:26:00
4,17850,22752.0,2,2010-12-04 08:26:00
7,17850,22633.0,6,2010-12-04 08:28:00
8,17850,22632.0,6,2010-12-03 08:28:00
...,...,...,...,...
272399,15249,23399.0,12,2011-10-08 11:37:00
272400,15249,22727.0,4,2011-10-08 11:37:00
272401,15249,23434.0,12,2011-10-08 11:37:00
272402,15249,23340.0,12,2011-10-07 11:37:00


In [8]:
#header=["userID", "itemID", "rating", "timestamp"]

df = df.rename(columns={

    'CustomerID':"userID",'StockCode':"itemID",'Quantity':"rating",'DeliveryDate':"timestamp"

})

df["userID"] = df["userID"].astype(int)
df["itemID"] = df["itemID"].astype(int)

##2. Split the data using the Spark chronological splitter provided in utilities

In [9]:
from recommenders.datasets.python_splitters import python_chrono_split

# Split the data: 75% train, 25% test (chronological split)
train, test = python_chrono_split(df, ratio=0.75)


In [10]:

train_file = "train.csv"
test_file = "test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)



In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185041 entries, 37126 to 137323
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   userID     185041 non-null  int64         
 1   itemID     185041 non-null  int64         
 2   rating     185041 non-null  int64         
 3   timestamp  185041 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 7.1 MB


##3. Train the NCF model on the training data, and get the top-k recommendations for our testing data
NCF accepts implicit feedback and generates prospensity of items to be recommended to users in the scale of 0 to 1. A recommended item list can then be generated based on the scores. Note that this quickstart notebook is using a smaller number of epochs to reduce time for training. As a consequence, the model performance will be slighlty deteriorated.

In [12]:
from recommenders.models.ncf.dataset import Dataset as NCFDataset


data = NCFDataset(train_file="train.csv", test_file="test.csv", seed=SEED)


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
 29%|██▊       | 1001/3498 [00:22<00:46, 53.77it/s]/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' i

In [18]:
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.utils.timer import Timer


model = NCF(
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16, 8, 4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)


In [19]:
with Timer() as train_time:
    model.fit(data)

print("Training completed in {:.2f} seconds.".format(train_time.interval))


Training completed in 123.38 seconds.


In [21]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 24.1455 seconds for prediction.


##4. Evaluate how well NCF performs
The ranking metrics are used for evaluation.

In [22]:
from recommenders.evaluation.python_evaluation import (
    map_at_k, ndcg_at_k, precision_at_k, recall_at_k
)

eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print(f"MAP:\t\t{eval_map:.6f}")
print(f"NDCG:\t\t{eval_ndcg:.6f}")
print(f"Precision@K:\t{eval_precision:.6f}")
print(f"Recall@K:\t{eval_recall:.6f}")


MAP:		0.031479
NDCG:		0.071343
Precision@K:	0.048571
Recall@K:	0.051730


In [ ]:
#hyperparameter tuning

In [25]:
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

results = []

factors_list = [4, 8]
layer_configs = [[16, 8, 4], [32, 16, 8]]
model_types = ["NeuMF", "GMF", "MLP"]

for model_type in model_types:
    for n_factors in factors_list:
        for layer_sizes in layer_configs:

            if model_type == "GMF":
                layer_sizes = [0]

            print(f"\nTraining model: {model_type}, factors: {n_factors}, layers: {layer_sizes}")
            model = NCF(
                n_users=data.n_users,
                n_items=data.n_items,
                model_type=model_type,
                n_factors=n_factors,
                layer_sizes=layer_sizes,
                n_epochs=5,
                batch_size=256,
                learning_rate=1e-3,
                verbose=0,
                seed=42
            )
            model.fit(data)

            # Uses a sample of 300 users for faster prediction as it took foerevr
            sample_users = train.userID.unique()[:300]
            item_ids = list(train.itemID.unique())
            users, items, preds = [], [], []

            for user_id in sample_users:
                repeated_users = [user_id] * len(item_ids)
                users.extend(repeated_users)
                items.extend(item_ids)
                preds.extend(model.predict(repeated_users, item_ids, is_list=True))

            all_predictions = pd.DataFrame({
                "userID": users,
                "itemID": items,
                "prediction": preds
            })
            merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
            all_predictions = merged[merged.rating.isnull()].drop("rating", axis=1)

            # Evaluate
            map_score = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
            ndcg_score = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
            prec_score = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
            rec_score = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

            results.append({
                "Model": model_type,
                "Factors": n_factors,
                "Layers": str(layer_sizes),
                "MAP": round(map_score, 4),
                "NDCG": round(ndcg_score, 4),
                "Precision@K": round(prec_score, 4),
                "Recall@K": round(rec_score, 4)
            })

# Show results
import pandas as pd
results_df = pd.DataFrame(results)
results_df.sort_values(by="MAP", ascending=False)



Training model: NeuMF, factors: 4, layers: [16, 8, 4]


/usr/local/lib/python3.11/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '



Training model: NeuMF, factors: 4, layers: [32, 16, 8]


/usr/local/lib/python3.11/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '



Training model: NeuMF, factors: 8, layers: [16, 8, 4]


...


Training model: MLP, factors: 8, layers: [16, 8, 4]


/usr/local/lib/python3.11/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '



Training model: MLP, factors: 8, layers: [32, 16, 8]


/usr/local/lib/python3.11/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


,Model,Factors,Layers,MAP,NDCG,Precision@K,Recall@K
3,NeuMF,8,"[32, 16, 8]",0.0482,0.0088,0.0742,0.0726
2,NeuMF,8,"[16, 8, 4]",0.0322,0.0060,0.0553,0.0543
1,NeuMF,4,"[32, 16, 8]",0.0320,0.0064,0.0590,0.0580
0,NeuMF,4,"[16, 8, 4]",0.0287,0.0058,0.0556,0.0513
11,MLP,8,"[32, 16, 8]",0.0245,0.0052,0.0488,0.0465
9,MLP,4,"[32, 16, 8]",0.0237,0.0051,0.0485,0.0427
7,GMF,8,[0],0.0223,0.0048,0.0444,0.0384
6,GMF,8,[0],0.0187,0.0042,0.0410,0.0325
10,MLP,8,"[16, 8, 4]",0.0170,0.0037,0.0349,0.0313
8,MLP,4,"[16, 8, 4]",0.0168,0.0038,0.0339,0.0300


In [27]:
EPOCHS = 50  # full training for final model

model = NCF(
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=8,
    layer_sizes=[32, 16, 8],
    n_epochs=EPOCHS,
    batch_size=256,
    learning_rate=1e-3,
    verbose=1,
    seed=42
)


In [28]:
from recommenders.utils.timer import Timer

with Timer() as train_time:
    model.fit(data)

print("Final model training complete in {:.2f} seconds.".format(train_time.interval))


Final model training complete in 1226.91 seconds.


In [29]:
#Generate Final Predictions for All Users
from recommenders.utils.timer import Timer

with Timer() as test_time:
    users, items, preds = [], [], []
    item_ids = list(train.itemID.unique())

    for user_id in train.userID.unique():
        repeated_users = [user_id] * len(item_ids)
        users.extend(repeated_users)
        items.extend(item_ids)
        preds.extend(model.predict(repeated_users, item_ids, is_list=True))

    all_predictions = pd.DataFrame({
        "userID": users,
        "itemID": items,
        "prediction": preds
    })


merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
all_predictions = merged[merged.rating.isnull()].drop("rating", axis=1)

print(" Prediction completed in {:.2f} seconds.".format(test_time.interval))


 Prediction completed in 19.24 seconds.


In [30]:
# Format Predication for Rec output
all_predictions = all_predictions[['userID', 'itemID', 'prediction']]
all_predictions = all_predictions.rename(columns={
    "userID": "CustomerID",
    "itemID": "StockCode",
    "prediction": "probability"
})


# Reccomndations

In [31]:
def recommend_product(customer_id):
    print(" \n---------- Top 5 Bought StockCodes -----------\n")
    top_5_bought = df_order[df_order['CustomerID'] == customer_id][['CustomerID', 'StockCode', 'Quantity']].nlargest(5, 'Quantity')
    print(top_5_bought)

    print('\n-------Product Name of Bought StockCodes ------\n')
    print(df_product[df_product.StockCode.isin(top_5_bought.StockCode)]['Product Name'])

    print("\n --------- Top 5 Recommendations ------------ \n")
    recommend = all_predictions[all_predictions['CustomerID'] == customer_id].nlargest(5, 'probability')
    print(recommend)

    print('\n-------Product Name of Recommendations ------\n')
    print(df_product[df_product.StockCode.isin(recommend.StockCode)]['Product Name'])


In [34]:
df_order = pd.read_excel('Rec_sys_data.xlsx', sheet_name='order')
df_customer = pd.read_excel('Rec_sys_data.xlsx', sheet_name='customer')
df_product = pd.read_excel('Rec_sys_data.xlsx', sheet_name='product')


In [35]:
recommend_product(13137)



 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
234414       13137     84077        48
234443       13137     23321        13
50797        13137     21985        12
234404       13137     22296        12
234418       13137     22297        12

-------Product Name of Bought StockCodes ------

70      MightySkins Skin Decal Wrap Compatible with Li...
490           Window Tint Film Mitsubishi (all doors) DIY
694     Harriton Men's Paradise Short-Sleeve Performan...
1065    MightySkins Skin For Samsung Galaxy J3 (2016),...
1339    MightySkins Skin Decal Wrap Compatible with Le...
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode  probability
1354297       13137      22383     0.986600
1354565       13137      22664     0.969105
1355666       13137      84991     0.959843
1353910       13137      21932     0.950002
1353911       13137      21933     0.934546

-------Product Name of Recom